In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os,sys
sys.path.append(os.path.abspath('./../../'))
from cell_extractor import utils 
from cell_extractor.diagnostics.ToolKit import get_DataFrame_from_query_result,find_equivalence,find_agreement,get_DataFrame_from_detection_df
from Controllers.MarkedCellController import MarkedCellController
from Controllers.SqlController import SqlController
import pandas as pd
from cell_extractor.CellAnnotationUtilities import CellAnnotationUtilities
from cell_extractor.AnnotationProximityTool import AnnotationProximityTool
from matplotlib.patches import Patch
from collections import Counter
from lib.UrlGenerator import UrlGenerator
from cell_extractor.QcGenerater import QcGenerater


In [25]:
animal = 'DK41'
controller = SqlController()
sections = controller.get_sections(animal,1)
file_names = np.array([i.file_name for i in sections])
repeating_sections = np.where(file_names[1:]==file_names[:-1])[0]
factor = controller.get_resolution(animal)


In [31]:
controller = MarkedCellController()
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':16}
round3_sures = get_DataFrame_from_query_result(controller.get_marked_cells(search_dict),'Sure',factor)
round3_sures = round3_sures.drop(round3_sures[[int(i) in repeating_sections for i in round3_sures.section]].index)
round3_sures_tool = find_equivalence(round3_sures)
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':17}
round3_unsures = get_DataFrame_from_query_result(controller.get_marked_cells(search_dict),'Unsure',factor)
round3_unsures = round3_unsures.drop(round3_unsures[[int(i) in repeating_sections for i in round3_unsures.section]].index)
round3_unsures_tool = find_equivalence(round3_unsures)
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':19}
connected = get_DataFrame_from_query_result(controller.get_marked_cells(search_dict),'Connected',factor)
connected = connected.drop(connected[[int(i) in repeating_sections for i in connected.section]].index)
connected_tool = find_equivalence(connected)
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':18}
false_negatives = get_DataFrame_from_query_result(controller.get_marked_cells(search_dict),'False_negative',factor)
false_negatives = false_negatives.drop(false_negatives[[int(i) in repeating_sections for i in false_negatives.section]].index)
false_negatives_tool = find_equivalence(false_negatives,distance=5)


calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 980
after removing duplicates 490
calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 8758
after removing duplicates 4379
calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 43
after removing duplicates 43
calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 886
after removing duplicates 751


In [4]:
disagree = [['Sure_Marissa_NEGATIVE', 'Sure_Julian_POSITIVE'],['Sure_Marissa_POSITIVE', 'Sure_Julian_NEGATIVE']]
round3_sure_disagree = find_annotation_in_category(round3_sures_tool,disagree)

In [5]:
disagree = [['Unsure_Marissa_POSITIVE', 'Unsure_Julian_NEGATIVE'],['Unsure_Marissa_NEGATIVE', 'Unsure_Julian_POSITIVE']]
round3_unsure_disagree = find_annotation_in_category(round3_unsures_tool,disagree)

In [6]:
agree = [['False_negative_Julian_POSITIVE', 'False_negative_Marissa_POSITIVE'],]
disagree = [['False_negative_Julian_POSITIVE'],['False_negative_Marissa_POSITIVE']]
false_negatives_agreed,false_negatives_disagreed = find_agreement(false_negatives_tool,agree,disagree)

In [7]:
round3_sure_disagree,round3_unsure_disagree,false_negatives_disagreed

,x,y,section,name
222,35472.9,18529,164,Sure_Marissa_NEGATIVE
223,40544,22547,165,Sure_Marissa_NEGATIVE
224,40302.2,23485,165,Sure_Marissa_NEGATIVE
229,40799.1,20920,180,Sure_Marissa_NEGATIVE
232,40452,21692,199,Sure_Marissa_NEGATIVE


In [26]:
sure_disagree = np.array([round3_sure_disagree.x,round3_sure_disagree.y,round3_sure_disagree.section]).T
unsure_disagree = np.array([round3_unsure_disagree.x,round3_unsure_disagree.y,round3_unsure_disagree.section]).T
false_negative_disagree = np.array([false_negatives_disagreed.x,false_negatives_disagreed.y,false_negatives_disagreed.section]).T

In [27]:
sure_cells = utils.numpy_to_json(sure_disagree,category = f'Round3_Sure')
unsure_cells = utils.numpy_to_json(unsure_disagree,category = f'Round3_Unsure')
false_negative_cells = utils.numpy_to_json(false_negative_disagree,color_hex='#1d66db',category = f'false_negative')
urlGen = UrlGenerator()
urlGen.add_stack_image(animal,channel=1)
urlGen.add_stack_image(animal,channel=3,color='green')
urlGen.add_annotation_layer('Sure',annotations = sure_cells)
urlGen.add_annotation_layer('Unsure',annotations = unsure_cells)
urlGen.add_annotation_layer('False Negative',annotations = false_negative_cells)
sc_id = urlGen.add_to_database('DK41 round3 disagreement',34)
print(f'Beth QC url id {animal} : https://activebrainatlas.ucsd.edu/ng/?id={sc_id}')

Beth QC url id DK41 : https://activebrainatlas.ucsd.edu/ng/?id=655
